In [ ]:
%pip install geoai-py smoothify

In [ ]:
#This script works well in Colab only

import geoai
import leafmap
import rasterio
from rasterio import features
import geopandas as gpd
from shapely.geometry import shape

in_tif = "/content/predictions_resunet_gnn_mosaic.tif" ## Binary output
out_geojson = "/content/predictions_resunet_gnn_mosaic.geojson"
smooth_output = "/content/playa_prediction_smoothed.geojson"

with rasterio.open(in_tif) as src:
    band1 = src.read(1)
    transform = src.transform
    crs = src.crs

# If your raster is strictly 0/1 this is fine.
# If it is probability (0..1) or 0..255, change threshold accordingly.
mask = band1 == 1

# Polygonize: extract polygons where mask is True
geoms = []
vals = []
for geom, val in features.shapes(band1, mask=mask, transform=transform):
    # val should be 1 for the foreground due to mask=mask
    geoms.append(shape(geom))
    vals.append(int(val))

gdf = gpd.GeoDataFrame({"class": vals}, geometry=geoms, crs=crs)

# Optional cleanup: dissolve all class=1 polygons into one (or few) multipart features
gdf = gdf[gdf["class"] == 1].dissolve(by="class").reset_index()

# Save GeoJSON
gdf.to_file(out_geojson, driver="GeoJSON")

# Path to your GeoJSON file
geojson_path = out_geojson

# Read GeoJSON into a GeoDataFrame
gdf = gpd.read_file(geojson_path)

gdf_exploded = gdf.explode(index_parts=False).reset_index(drop=True)

print(gdf_exploded.geom_type)

smoothed_gdf = geoai.smooth_vector(
    gdf_exploded, smooth_iterations=3, output_path=smooth_output
)


In [ ]:
smoothed_style = {"color": "blue", "fillOpacity": 0, "weight": 1}
m = leafmap.Map()
m.add_gdf(
    smoothed_gdf, style=smoothed_style, layer_name="Smoothed Vector", info_mode=None
)
m